In [33]:
import glob
import os
import os.path as osp
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image

from utils import convert_nii_3d, get_trainable_slices
from wrapper import OASIS
from split import split_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
scans_home = 'data/scans'
labels_file = 'data/OASIS3_MRID2Label_052918.csv'

In [37]:
train_filenames, val_filenames, test_filenames = split_data(scans_home, labels_file)

num labels is 2113
num filenames is 2193
num experiments is 1954
OAS30134_MR_d0080 ('data/scans/OAS30134_MR_d0080/anat1/sub-OAS30134_sess-d0080_acq-TSE_T2w.nii.gz', '0.0')
OAS30336_MR_d0012 ('data/scans/OAS30336_MR_d0012/anat1/sub-OAS30336_sess-d0012_acq-TSE_T2w.nii.gz', '0.0')
OAS30087_MR_d0260 ('data/scans/OAS30087_MR_d0260/anat1/sub-OAS30087_sess-d0260_acq-TSE_T2w.nii.gz', '0.5')
OAS30073_MR_d2851 ('data/scans/OAS30073_MR_d2851/anat1/sub-OAS30073_sess-d2851_acq-TSE_T2w.nii.gz', '0.0')
OAS31125_MR_d3093 ('data/scans/OAS31125_MR_d3093/anat2/sub-OAS31125_sess-d3093_acq-TSE_T2w.nii.gz', '0.0')


TypeError: 'NoneType' object is not iterable

In [5]:
print(len(train_filenames))
print(len(val_filenames))
print(len(test_filenames))

1072
134
134


In [7]:
train_dataset = OASIS('scans', train_filenames, preload = True)
val_dataset = OASIS('scans', val_filenames, preload = True)
test_dataset = OASIS('scans', test_filenames, preload = True)

finished preloading
finished preloading
finished preloading


In [10]:
image_array, label = train_dataset[4]
print(image_array.shape)
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))
save_path = 'test_preview_scan/'

(256, 256)
39720
4934
4865


In [35]:
# get np H, W, D image
img = convert_nii_3d('data/scans/OAS30001_MR_d0129/anat1/sub-OAS30001_sess-d0129_acq-TSE_T2w.nii.gz') #preview_scan
# img = convert_nii_3d('data/scans/OAS30001_MR_d2430/anat2/sub-OAS30001_sess-d2430_acq-TSE_T2w.nii.gz') #preview_scan_anat2_TSE
# img = convert_nii_3d('data/scans/OAS30001_MR_d0129/anat4/sub-OAS30001_sess-d0129_T2w.nii.gz') #preview_scan_non_TSE
print(img.shape)
# loop over indices and print out mean of slice
for i in range(img.shape[2]):
    print(i, img[:,:,i].shape, img[:,:,i].mean())

(256, 256, 36)
0 (256, 256) 18.835647583007812
1 (256, 256) 19.651214599609375
2 (256, 256) 18.936920166015625
3 (256, 256) 17.616287231445312
4 (256, 256) 17.561676025390625
5 (256, 256) 17.472305297851562
6 (256, 256) 16.920806884765625
7 (256, 256) 16.721939086914062
8 (256, 256) 17.168563842773438
9 (256, 256) 17.397903442382812
10 (256, 256) 17.559188842773438
11 (256, 256) 17.627609252929688
12 (256, 256) 17.915206909179688
13 (256, 256) 18.420913696289062
14 (256, 256) 19.616134643554688
15 (256, 256) 19.139511108398438
16 (256, 256) 18.763336181640625
17 (256, 256) 17.83941650390625
18 (256, 256) 17.493011474609375
19 (256, 256) 17.338897705078125
20 (256, 256) 17.54119873046875
21 (256, 256) 17.26580810546875
22 (256, 256) 17.243606567382812
23 (256, 256) 16.181503295898438
24 (256, 256) 16.340194702148438
25 (256, 256) 16.035018920898438
26 (256, 256) 15.744766235351562
27 (256, 256) 14.8021240234375
28 (256, 256) 14.424560546875
29 (256, 256) 13.15753173828125
30 (256, 256) 

In [10]:
# print out mean of all slices using np.mean (axis)
print(img.shape)
print(img.reshape(-1, img.shape[2]).mean(axis=0))

(256, 256, 36)
[18.83564758 19.6512146  18.93692017 17.61628723 17.56167603 17.4723053
 16.92080688 16.72193909 17.16856384 17.39790344 17.55918884 17.62760925
 17.91520691 18.4209137  19.61613464 19.13951111 18.76333618 17.8394165
 17.49301147 17.33889771 17.54119873 17.26580811 17.24360657 16.1815033
 16.3401947  16.03501892 15.74476624 14.80212402 14.42456055 13.15753174
 12.49742126 11.61732483 10.76405334  7.98591614  5.83325195  3.88075256]


In [19]:
#print out max mean of all slices using mean max
print(img.reshape(-1, img.shape[2]).mean(axis=0).max())

32.42316228693182


In [41]:
# get all images that are above a threshold
THRESHOLD_VAL = 0.75
threshold = img.reshape(-1, img.shape[2]).mean(axis=0).max() * THRESHOLD_VAL
print(np.arange(img.shape[2])[img.reshape(-1, img.shape[2]).mean(axis=0) > threshold])

[ 93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146
 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162]


In [36]:
print(np.arange(img.shape[2])[get_trainable_slices(img)])

[10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28]
